Chatbot
=======

Create a conversational AI using multi-shot prompting.

Conversational Assistants are a common use case for Gen AI, and the latest frontier models are really good at nuanced conversation. Gradio is used to great a simple user interface and prompting is used to provide context, information and examples.

The system prompt is used to give context on the business, and set the tone for the LLM.

# Dependencies

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Setup

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

# Functions and Prompts

## System prompt used to prime LLM

OpenAI format for messages:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

In [ ]:
system_message = "You are a helpful assistant"

## Chat function

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# UI for Chatbot

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

# Multi-Shot Prompting

## System message designed to upsell with latest promos

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## Add example of how to respond in specific scenarios

And really annoy people like me, although I do like a good hat and who needs new shoes when german paratrooper boots from army surplus last from decades?

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## Dynamically update system prompt based on keywords in user messages

In [3]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()